# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

### Parameters
batch size = 30 <br>
epoch size = 25


In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
# tf.random.set_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
# train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
# val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())

batch_size = 30 # experiment with the batch size

In [4]:
len(train_doc)

663

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
#source_path is the path where the image folders are
#folder_list is the .csv files (train_doc or val_doc)
def generator(source_path, folder_list, batch_size): 
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    x = 30
    y = 100
    z = 100
    channels = 3
    #can choose the number of images to use for 1 video - 30 may increase the training time
    img_idx = range(0,30,3) #create a list of image numbers you want to use for a particular video
    #while True: makes sure that code execution flow will not stop - for epochs
    while True:
        t = np.random.permutation(folder_list) #shuffles the folders - so there is a mix of all the classes
        #num_batches is the no.of sequences(video)/batch_size
        num_batches = np.math.floor(len(folder_list)/batch_size) # calculate the number of batches
        for batch in range(num_batches): 
            batch_data = np.zeros((batch_size,len(img_idx),y,z,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    image = imresize(image, (100,100))
                      
                    batch_data[folder,idx,:,:,0] = (image[:, :, 0] - image[:, :, 0].min())/(image[:, :, 0].max() - image[:, :, 0].min())
                    batch_data[folder,idx,:,:,1] = (image[:, :, 1] - image[:, :, 1].min())/(image[:, :, 1].max() - image[:, :, 1].min())
                    batch_data[folder,idx,:,:,2] = (image[:, :, 2] - image[:, :, 2].min())/(image[:, :, 2].max() - image[:, :, 2].min())
                                        
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        # The data is left as is, instead of adding more data to match the batch size. 
        # This is done, on the understanding/assumption that due to the random sampling at the start of each epoch, the data would be trained across the other epochs. 
        # This helps to add an unknown data while training, which can help reduce the chance of overfitting


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/notebooks/storage/Final_data/Collated_training/train'
val_path = '/notebooks/storage/Final_data/Collated_training/val'
# train_path = './Project_data/train'
# val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 25
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 25


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model = Sequential()

model.add(Conv3D(32, kernel_size=(3,3,3),activation='relu',input_shape = (10, 100, 100, 3)))
model.add(MaxPooling3D(pool_size=(2, 2,2)))
model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3,3,3),activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2,2)))
model.add(Dropout(0.25))

model.add(Conv3D(32, kernel_size=(3,3,3),activation='relu', padding='same'))
model.add(MaxPooling3D(pool_size=(1,1,1)))
model.add(Dropout(0.25))

# flatten and put a fully connected layer
model.add(Flatten())
model.add(Dense(128, activation='relu')) # fully connected
model.add(Dropout(0.5))

# softmax layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 8, 98, 98, 32)     2624      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 4, 49, 49, 32)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 49, 49, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 2, 47, 47, 64)     55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 1, 23, 23, 64)     0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 23, 23, 64)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 23, 23, 32)     55328     
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

#Reducing the learning rate - change the learning rate everytime the validation accuracy becomes steady
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=2, min_lr=0.001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
#num_epochs are typically 50 or 100, based on the learning rate
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 30
Source path =  ./Project_data/train ; batch size = 30
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 32s 1s/step - loss: 1.6208 - categorical_accuracy: 0.2246 - val_loss: 1.6033 - val_categorical_accuracy: 0.2417

Epoch 00001: saving model to model_init_2019-12-2218_59_47.526851/model-00001-1.62084-0.22464-1.60332-0.24167.h5
Epoch 2/25
23/23 [==============================] - 28s 1s/step - loss: 1.5811 - categorical_accuracy: 0.2623 - val_loss: 1.5223 - val_categorical_accuracy: 0.2833

Epoch 00002: saving model to model_init_2019-12-2218_59_47.526851/model-00002-1.58113-0.26232-1.52227-0.28333.h5
Epoch 3/25
23/23 [==============================] - 27s 1s/step - loss: 1.4717 - categorical_accuracy: 0.3420 - val_loss: 1.3977 - val_categorical_accuracy: 0.4083

Epoch 00003: saving model to model_init_2019-12-2218_59_47.526851/model-00003-1.47173-0.34203-1.39765-0.40833.h5
Epoch 4/25
23/23 [==============================] - 27s 1s/step - loss: 1.3454 - categorical_accuracy: 0.4348 - val_loss: 1.3880 - val_categorical_accuracy: 0.5000

Epoch 00004: